In [2]:
from azureml.core import Workspace, ScriptRunConfig, Datastore, Dataset
from azureml.core.compute import ComputeTarget

In [3]:
ws= Workspace.from_config("./config")

In [4]:
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_selected_columns")
az_default_store= ws.get_default_datastore()

In [40]:
from azureml.core import LinkedService, SynapseWorkspaceLinkedServiceConfiguration
synapse_link_config= SynapseWorkspaceLinkedServiceConfiguration(
    subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad",
    resource_group="RG-MSFT-ML-POC-01",
    name="synapse-msft-ml-poc-01"   # we have to take synapse workspace name

)
linked_service = LinkedService.register(
    workspace=ws,
    name='synapse-link1',
    linked_service_config=synapse_link_config
)

In [5]:
from azureml.core import LinkedService

linked_service = LinkedService.get(ws, 'synapse-link1')

Class LinkedService: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [6]:
ws.linked_services

Class LinkedService: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'synapse-link1': LinkedService(workspace=Workspace.create(name='aml-msft-ml-poc1', subscription_id='a4df42c9-c1d2-44c4-b2c2-05edb4b27aad', resource_group='RG-MSFT-ML-POC-01'), name=synapse-link1, type=LinkedServiceLinkType.synapse, linked_service_resource_id=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.Synapse/workspaces/synapse-msft-ml-poc-01, system_assigned_identity_principal_id=939e1162-b666-4cd3-bc98-c0ef71edda66}

In [7]:
from azureml.core.compute import SynapseCompute, ComputeTarget
spark_pool_name= "msazuremlpoc"     #we have to take spark pool compute name
attached_synapse_name= "microsoftsynapse"
attach_config = SynapseCompute.attach_configuration(
    linked_service=linked_service,
    type="SynapseSpark",
    pool_name=spark_pool_name
)
synapse_compute= ComputeTarget.attach(
    workspace=ws,
    name=attached_synapse_name,
    attach_configuration=attach_config
)
synapse_compute.wait_for_completion(True)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


SucceededProvisioning operation finished, operation "Succeeded"


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [8]:
print(linked_service.system_assigned_identity_principal_id)

939e1162-b666-4cd3-bc98-c0ef71edda66


In [9]:
cluster= ws.compute_targets[attached_synapse_name]

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [36]:
cluster

id: /subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/aml-msft-ml-poc1/computes/microsoftsynapse,
name: microsoftsynapse,
tags: None,
location: eastus,
properties: {'computeType': 'SynapseSpark', 'computeLocation': 'eastus', 'description': None, 'resourceId': '/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.Synapse/workspaces/synapse-msft-ml-poc-01/bigDataPools/msazuremlpoc', 'provisioningErrors': None, 'provisioningState': 'Succeeded', 'properties': {'sparkVersion': '3.2', 'nodeCount': 10, 'nodeSizeFamily': 'MemoryOptimized', 'nodeSize': 'Medium', 'autoScaleProperties': {'enabled': True, 'maxNodeCount': 3, 'minNodeCount': 3}, 'autoPauseProperties': {'enabled': True, 'delayInMinutes': 15}}}

In [46]:
synapse_compute

id: /subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/aml-msft-ml-poc1/computes/microsoftsynapse,
name: microsoftsynapse,
tags: None,
location: eastus,
properties: {'computeType': 'SynapseSpark', 'computeLocation': 'eastus', 'description': None, 'resourceId': '/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.Synapse/workspaces/synapse-msft-ml-poc-01/bigDataPools/msazuremlpoc', 'provisioningErrors': None, 'provisioningState': 'Succeeded', 'properties': {'sparkVersion': '3.2', 'nodeCount': 10, 'nodeSizeFamily': 'MemoryOptimized', 'nodeSize': 'Medium', 'autoScaleProperties': {'enabled': True, 'maxNodeCount': 3, 'minNodeCount': 3}, 'autoPauseProperties': {'enabled': True, 'delayInMinutes': 15}}}

In [10]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [11]:
from azureml.core.environment import CondaDependencies
myenv=CondaDependencies()
myenv.add_conda_package("azure-core==1.20.0")

In [12]:
from azureml.core.runconfig import RunConfiguration 
run_config = RunConfiguration(framework="pyspark")
run_config.target= synapse_compute
run_config.environment = myenv 
run_config.spark.configuration["spark.driver.memory"]="1g"
run_config.spark.configuration["spark.driver.cores"]=2
run_config.spark.configuration["spark.executor.memory"]="1g"
run_config.spark.configuration["spark.executor.cores"]=1
run_config.spark.configuration["spark.executor.instances"]=1

In [13]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_synapse")

In [50]:
from azureml.pipeline.steps import PythonScriptStep 
from azureml.pipeline.core import PipelineData 
input_ds= ws.datasets.get("Titanic_selected_columns")
datafolder= PipelineData('datafolder', datastore=ws.get_default_datastore())

In [14]:
from azureml.pipeline.steps import PythonScriptStep, SynapseSparkStep
from azureml.core.environment import Environment
env = Environment(name="myenv")
env.python.conda_dependencies.add_pip_package("azureml-core==1.20.0")

step_1 = SynapseSparkStep(name = 'synapse-spark',
                          file = 'Singlestep_Training.py',
                          source_directory="Training_python_scripts/", 
                          compute_target = synapse_compute,
                          driver_memory = "7g",
                          driver_cores = 4,
                          executor_memory = "7g",
                          executor_cores = 2,
                          num_executors = 1,
                          environment = env)

Class SynapseSparkStep: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


only conda_dependencies specified in environment will be used in Synapse Spark run.


In [15]:
steps=[step_1]
from azureml.pipeline.core import Pipeline 
new_pipeline = Pipeline(workspace=ws, steps=steps)

In [16]:
new_experiment= Experiment(workspace=ws, name="pipelineexp01")
pipeline_exp_run= new_experiment.submit(new_pipeline)
pipeline_exp_run.wait_for_completion(show_output=True)

Created step synapse-spark [a6d0b633][514426c2-e8d7-493e-91f7-08dc46741dba], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun eee4575c-ffdd-47e1-a626-8381f9285b8f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/eee4575c-ffdd-47e1-a626-8381f9285b8f?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73
PipelineRunId: eee4575c-ffdd-47e1-a626-8381f9285b8f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/eee4575c-ffdd-47e1-a626-8381f9285b8f?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73
PipelineRun Status: Running


StepRunId: 13f3a7e0-9914-4e29-99ae-cf5ef2f357eb
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/13f3a7e0-9914-4e29-99ae-cf5ef2f357eb?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Service invocation failed!\r\nRequest: POST account-rp.vienna-eastus.svc/rp/workspaces/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1/listResourceKeys\r\nStatus Code: 400 BadRequest\r\nError Code: ValidationError\r\nReason Phrase: Not Found response body: {\"error\":{\"code\":\"ResourceNotFound\",\"message\":\"The Resource 'Microsoft.insi\r\nResponse Body: {\n  \"error\": {\n    \"code\": \"ValidationError\",\n    \"severity\": null,\n    \"message\": \"Not Found response body: {\\\"error\\\":{\\\"code\\\":\\\"ResourceNotFound\\\",\\\"message\\\":\\\"The Resource 'Microsoft.insights/components/appinsamlmsftmlpoc119092022' under resource group 'RG-MSFT-ML-POC-01' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix\\\"}}\",\n    \"messageFormat\": null,

In [51]:
dataprep_step3= PythonScriptStep(name="Datapreparation",
                                source_directory="Training_python_scripts/",
                                script_name= "Dataprep_pipeline.py",
                                inputs=[input_ds.as_named_input('raw_data')],
                                outputs=[datafolder],
                                runconfig=run_config,
                                arguments=['--datafolder', datafolder])

In [52]:
train_step3= PythonScriptStep(name="ModelTraining",
                             source_directory="Training_python_scripts/",
                             script_name="Training_pipeline_explain.py",
                             inputs=[datafolder],
                             runconfig=run_config,
                             arguments=['--datafolder',datafolder])

In [53]:
steps= [dataprep_step3,train_step3]

In [54]:
from azureml.pipeline.core import Pipeline 
new_pipeline = Pipeline(workspace=ws, steps=steps)

In [55]:
new_experiment= Experiment(workspace=ws, name="pipelineexp01")
pipeline_exp_run= new_experiment.submit(new_pipeline)
pipeline_exp_run.wait_for_completion(show_output=True)

Created step Datapreparation [4225c369][b09e6dcb-9c43-4913-a6ef-cbead9d2c73d], (This step is eligible to reuse a previous run's output)
Created step ModelTraining [195cb667][492cce95-8a3b-462c-adb3-3ef7d9a92b21], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun c183d196-f09b-4403-a8f5-a405b7762216
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c183d196-f09b-4403-a8f5-a405b7762216?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73
PipelineRunId: c183d196-f09b-4403-a8f5-a405b7762216
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c183d196-f09b-4403-a8f5-a405b7762216?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73

PipelineRun Execution Summary
PipelineRun Status: Failed


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "messageFormat": "Pipeline has some failed steps. {0}",
        "messageParameters": {},
        "referenceCode": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2022-11-04T15:20:16.748737Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Pipeline has some failed steps. See child run or execution logs for more details.\",\n        \"messageFormat\": \"Pipeline has some failed steps. {0}\",\n        \"messageParameters\": {},\n        \"referenceCode\": \"PipelineHasStepJobFailed\",\n        \"details\": []\n    },\n    \"environment\